# Tutorial: Use a trained RNN in grainLearning calibration process.

## ⚙️ Install grainlearning package
(Not necessary if you are running jupyter-lab on an environment where grainlearning and rnn dependencies are installed)

pip install grainlearning --extras "rnn"

## Introduction to this tutorial
We hope that now you are familiar with grainLearning and the RNN module. In this tutorial we are going to explore how can we use a pretrained neural network as a surrogate model in the grainLearning calibration process. In such context, the RNN plays the role of a `DynamicSystem`.

We considered the case of *Triaxial Compression* DEM simulations. Such results have been used to trained an RNN and we will pick one of the examples in this dataset to show-case how the inference could be done.

This tutorial has three main parts:

1. Prepare the pre-trained model.
2. Create a callback function to link to `DynamicSystem`.
3. GrainLearning calibration loop.

Let's start importing some useful packages:

In [4]:
import numpy as np
import tensorflow as tf

from grainlearning import BayesianCalibration
import grainlearning.dynamic_systems
import grainlearning.rnn.predict as predict_rnn
import grainlearning.rnn.preprocessing as preprocessing_rnn

## 1. Prepare the pre-trained model 🗄️
For the purpose of this tutorial we are going to take an example from the a dataset containing DEM simulation results of a triaxial compression. 
Our RNN model was trained in a similar database, and we are going to take one example to infer from. In practice, such example could be the results of real-world experiments for wich we do not know the DEM _contact parameters_.

In [5]:
path_to_trained_model = '/Users/luisaorozco/Documents/Projects/GrainLearning/grainLearning/grainlearning/rnn/trained_models/rnn_triaxial_undrained'
path_to_data = '/Users/luisaorozco/Documents/Projects/GrainLearning/data/TriaxialCompression/triaxial_compression_variable_input.hdf5'

In [ ]:
# Load the pretrained model
model_rnn, train_stats, config_rnn = predict_rnn.get_pretrained_model(path_to_trained_model)

In [7]:
# Load a dataset to prepare synthetic data to use for the calibration
config_data_preparation = config_rnn.copy() # deeep copy to avoid modifying the config used to train.
config_data_preparation['raw_data'] = path_to_data
config_data_preparation['standardize_outputs'] = False # Necessay because we will get the observation from this dataset
data, _ = preprocessing_rnn.prepare_datasets(**config_data_preparation)

In [ ]:
# getting a single sample from the test dataset: Synthetic data
inputs, labels = next(iter(data['test'].batch(1)))
load_sequence = inputs['load_sequence'][0].numpy() # sequence input for the rnn, getting rid of 1st dimension: sample
contact_params = inputs['contact_parameters'][0].numpy() # The parameters inferred by GL should by as close as possible to these

`extra_contact_parameters` are not in `system.param_data` i.e. they are not parameters that need to be inferred. 
However, these are control parameters necessary to predict from RNN that are added at the end of  `contact_params`.
Given the values of `add_e0`, `add_pressure` and `add_experiment_type` in `config`, we can determine how many parmeters are extra.

In [9]:
add_pressure = True if 'add_pressure' not in config_rnn else config_rnn['add_pressure']
add_experiment_type = True if 'add_experiment_type' not in config_rnn else config_rnn['add_experiment_type']
add_e0 = False if 'add_e0' not in config_rnn else config_rnn['add_e0']

num_extra_contact_params = sum([add_pressure, add_experiment_type, add_e0])
extra_contact_params = contact_params[-num_extra_contact_params:]
contact_params = contact_params[:-num_extra_contact_params]

If no padding was taking into account during training, the predicitions of the RNN are going to be one `window_size` shorter.
We adapt the labels so that during the calibration we compare tensors with the same dimensions.

In [10]:
if 'pad_length' not in config_rnn or config_rnn['pad_length'] == 0: # no padding
    labels = labels[0, config_rnn['window_size']:, :].numpy()

In Grainlearning, the temporal dimension is always at the end, we need to switch the access of the data to comply with this format.

In [11]:
# convert arrays from [num_time_steps, num_features] -> [num_features, num_time_steps]
labels = np.moveaxis(labels, 0, -1)
load_sequence = np.moveaxis(load_sequence, 0, -1)

In this preparation steps we got few important arrays to work with:
- `load_sequence`: control of the system
- `labels`: observation (Synthetic data)
- `contact_params`: The parameters used to create the example. GrainLearning should be able to infer parameters close to these ones.
- `config_rnn`: dictionary containing the configration of the pre-trained RNN model.

## 2. Create a callback function to link to `DynamicSystem`

In [21]:
def predict_with_RNN(system, **_):
    window_size = config_rnn['window_size']
    sequence_length = np.shape(system.ctrl_data)[-1]
    
    # For compatibility with the RNN (first dimension is sample)
    load_sequence = np.moveaxis(system.ctrl_data, 0, -1)
    load_sequence = np.repeat(load_sequence[np.newaxis, :], system.num_samples, axis=0)
    
    # add extra_contact_params to the contct_params used to draw a prediction
    contact_params = np.array([np.append(i, extra_contact_params) for i in system.param_data])

    # Option 1
    data_inputs = ({'load_sequence': load_sequence, 'contact_parameters': contact_params}, load_sequence)
    data_inputs = tf.data.Dataset.from_tensor_slices(data_inputs)
    predictions = predict_rnn.predict_macroscopics(model_rnn, 
                                                   data_inputs, 
                                                   train_stats, 
                                                   config_rnn, 
                                                   batch_size=system.num_samples)
    
    # converting the predictions to GL format (temporal dimension at the end)
    predictions = np.moveaxis(predictions.numpy(), 1, -1) 
    system.set_sim_data(np.array(predictions))

## 3. GrainLearning calibration loop 🔁

In [24]:
def grainlearning_calibration(inputs: np.array, labels: np.array):
    """
    Main function defining and driving the grainLearning calibration.
    1. Define the BayesianCalibration and all its elements: DynamicSystem, inferences and ssampling parameters.
    2. Run the calibration and finally return the inferred parameters.

    :param inputs: numpy.array with the control data.
      In this example (undrained triaxial compression) a time sequence of the three principal strains.
    :param labels: numpy.array with the observation data.
      In this example a time sequence with macroscopic observables such as e, p, q.
    :return: most_prob_params numpy.array containing the inferred values of the parameters.
    """
    curr_iter = 0
    calibration = grainlearning.BayesianCalibration.from_dict(
        {
            "curr_iter": curr_iter,
            "num_iter": 10,
            "system": {
                "system_type": grainlearning.dynamic_systems.DynamicSystem, # because I'm not reading files, my data is generated by an RNN.
                "obs_data": labels,    # Synthetic data
                "obs_names": ['e', 'p', 'q'],
                "ctrl_data": load_sequence, # Synthetic data.
                "ctrl_name": ['e_z'],  # Only one of the strains (axial)
                "num_samples": 50,     # num of samples (gaussian mixture) generated per iteration
                "sim_name": 'Triaxial compression with RNN.',
                # to get these labels I opened the hdf5 file and query: file.attrs['contact_params']
                "param_names": ['E', 'v', 'kr', 'eta', 'mu'],
                "param_min": [7.00195859, 1.6e-4, 8.3e-4, 1.5e-4, 4.5e-2], # using the range of contact params used to train the RNN.
                "param_max": [9.99607979, 0.49952, 0.99875, 0.99878, 59.955],
                #"inv_obs_weight": [1, 1, 0.01],
                "callback": predict_with_RNN
            },
            "calibration": {
                "inference": {"ess_target": 0.3},
                "sampling": {
                "max_num_components": 5,
                "prior_weight": 0.01,
                },
            },
            "save_fig": -1, # Not generating plots
        }
    )

    print("Run the calibration")
    calibration.run()
    print("Calibration finished")
    return calibration.get_most_prob_params()


## 🎛️ Start the calibration

In [25]:
# Run the calibration
most_prob_params = grainlearning_calibration(load_sequence, labels)

Run the calibration
Bayesian calibration iter No. 0
Bayesian calibration iter No. 1
Bayesian calibration iter No. 2
Bayesian calibration iter No. 3
Bayesian calibration iter No. 4
Bayesian calibration iter No. 5
Bayesian calibration iter No. 6
Bayesian calibration iter No. 7
Bayesian calibration iter No. 8
Bayesian calibration iter No. 9


## ⚖️ Compare the results

In [27]:
# get the contact parameter that were supossed to be inferred, the rest were only necessary for drawing predictions with the RNN.
contact_params_reference = contact_params[:len(most_prob_params)]
print(f"Contact params (ground truth): {contact_params_reference} \nContact parameters calibrated via GL: {most_prob_params}")

Contact params (ground truth): [ 8.46679943  0.31648     0.6289      0.82335    50.808     ] 
Contact parameters calibrated via GL: [ 8.45177365  0.46319715  0.52777514  0.5096332  45.8883778 ]


In [32]:
# get the percentage error
print(f"error = {np.abs(most_prob_params - contact_params_reference) / contact_params_reference}")

error = [0.00177467 0.46359059 0.1607964  0.38102483 0.09682771]


📉 Also, if `save_fig` was set to 1, you can take a look at the generared plots for each one of the calibration iterations. 

## ✏️ Final tips

- Check always the dimensions and the order of the parameters in the tensors that you are using. 
- Keep in mind that in GrainLearning the last dimension is the time.